In [1]:
import site
site.addsitedir("/opt/nvidia/nsight-compute/2023.1.1/extras/python")
site.addsitedir("/opt/nvidia/nsight-compute/2023.1.1/sections")

In [2]:
import ncu_report
from ncu_report import IMetric
import pprint

In [3]:
report = ncu_report.load_report(f"A100_only_attention_prompt1_iter1_step5_replay_kernel_huggingface/A100_only_attention_prompt1_iter1_step5_replay_kernel_huggingface.ncu-rep")

In [4]:
kernels = report[0]
metric_kernel = kernels[0]
metrics = [metric_kernel[name] for name in metric_kernel]

In [5]:
# yes_list = [("1_Qu","2_Ke","3_Va"),("5-1_MatMul_Query_Key_Then_Scale"),("5-2_Softmax_score"),("5-3_MatMul_Softmax_Value"),("7_Out_Linear"),("1_Projection"),("Linear"),("1_Q","2_K","3_V","5-1_MatMul_Query_Key_Then_Scale", "5-2_Softmax_score","5-3_MatMul_Softmax_Value","7_Out_Linear","1_Projection","Linear")]
# yes_list = [("1_Qu","2_Ke","3_Va"),("5-1_MatMul_Query_Key_Then_Scale"),("5-2_Softmax_score"),("5-3_MatMul_Softmax_Value"),("7_Out_Linear"),("1_Projection"),("Linear")]
# yes_list = [("1_Q","2_K","3_V","5-1_MatMul_Query_Key_Then_Scale", "5-2_Softmax_score","5-3_MatMul_Softmax_Value","7_Out_Linear","1_Projection","Linear")]
# yes_list = ("1_Q","2_K","3_V","5-1_MatMul_Query_Key_Then_Scale", "5-2_Softmax_score","5-3_MatMul_Softmax_Value","7_Out_Linear","1_Projection","Linear")
yes_list = ("1_Q","2_K","3_V","5-1_MatMul_Query_Key_Then_Scale","5-2_Softmax","5-3_MatMul_Softmax_Value","7_Out_Linear","1_GEGLU","Linear_")
no_list = ("splitKreduce_kernel")

In [6]:
kernel = kernels[13]
print(kernel.name())

ampere_sgemm_128x128_tn


In [7]:
nvtx_info = kernel.nvtx_state()
print(nvtx_info[1],nvtx_info[2]) # nvtx_info[0] : default domain, nvtx_info[1] : ncu_prof, nvtx_info[2] : nsys_prof
nvtx_domain_ncu_prof = kernel.nvtx_state()[1]
nvtx_domain_nsys_prof = kernel.nvtx_state()[2]

ncu_prof nsys_prof


In [8]:
print(nvtx_domain_nsys_prof.push_pop_ranges())
print(nvtx_domain_ncu_prof.start_end_ranges())

('denoising network', 'denoising_step_2', 'noise_prediction_2', 'U-Net', 'contracting_path', '1_CrossAttnDownBlock2D_input_NCHW_2_320_64_64_embedding_2_77_768', 'Transformer2DModel_head(num/dim)8/40_ch(in/out)320/320', 'transformer_computation', 'transformer_BasicTransformerBlock', '2_self_attn_Attention', '5_Scaled Dot Product Attention_Q16_4096_40_K16_4096_40', '5-1_MatMul_Query_Key_Then_Scale_Q16_4096_40_KT16_40_4096')
('denoiser_step_2', 'unet_block_1', 'attn_block_2')


In [9]:
Execution_Time_t=0
Executed_Inst_t=0
Total_Traffic_t=0

for kernel in kernels:
    # range_name = nvtx_info.push_pop_ranges()[6] # 6 Resnet/Transformer Block Level
    nvtx_domain_ncu_prof = kernel.nvtx_state()[1]
    nvtx_domain_nsys_prof = kernel.nvtx_state()[2]
    ncu_range_name = nvtx_domain_ncu_prof.start_end_ranges()[1]
    ncu_range_name_1 = nvtx_domain_ncu_prof.start_end_ranges()[-1]
    nsys_range_name = nvtx_domain_nsys_prof.push_pop_ranges()[-1]
    range_names = ["unet_block_1","unet_block_2","unet_block_3","unet_block_4","unet_block_5","unet_block_6","unet_block_7","unet_block_8","unet_block_9"]
    if ncu_range_name in range_names:
        # print(range_name)
        if nsys_range_name.startswith(yes_list) :
            if not kernel.name().startswith(no_list) :
                Work_Achieved = (kernel["smsp__sass_thread_inst_executed_op_fadd_pred_on.sum.per_cycle_elapsed"].value()\
                                + kernel["smsp__sass_thread_inst_executed_op_fmul_pred_on.sum.per_cycle_elapsed"].value()\
                                + kernel["derived__smsp__sass_thread_inst_executed_op_ffma_pred_on_x2"].value()) \
                                * kernel["smsp__cycles_elapsed.avg.per_second"].value()
                Traffic_Achieved = kernel["dram__bytes.sum.per_second"].value()                
                inst = kernel["thread_inst_executed"].value()
                inst2 = kernel["inst_executed"].value()
                Execution_Time = kernel["gpu__time_duration.sum"].value()
                Executed_Inst = Work_Achieved * Execution_Time * 0.000000001
                Total_Traffic = Traffic_Achieved * Execution_Time * 0.000000001
                Execution_Time_t += Execution_Time
                Executed_Inst_t += Executed_Inst
                Total_Traffic_t += Total_Traffic
                
                Work_Achieved = Work_Achieved/(1000000000000)
                Traffic_Achieved = Traffic_Achieved/(1000000000)
                
                SM_BUSY = kernel["sm__throughput.avg.pct_of_peak_sustained_elapsed"].value()
                MEM_BUSY = kernel["gpu__compute_memory_throughput.avg.pct_of_peak_sustained_elapsed"].value()
                # MEM_BUSY = kernel["gpu__compute_memory_access_throughput.avg.pct_of_peak_sustained_elapsed"].value()
                
                # print(f"{nsys_range_name} {kernel.name()} {Executed_Inst:.1f} {Total_Traffic/(1000*1000):.2f} , {Execution_Time/1000000}")
                print(f"{ncu_range_name} {nsys_range_name} {kernel.name()} {Execution_Time/1000:.1f} {Executed_Inst:.1E} {Total_Traffic/(1000*1000):.1f} {Work_Achieved:.1f} {Traffic_Achieved:.1f} {SM_BUSY:.1f} {MEM_BUSY:.1f}")
                # print(f"{Execution_Time_t:.2E}")
                # print(f"{Executed_Inst_t:.2E}")
                # print(f"{Total_Traffic_t:.2E}")

unet_block_1 1_Query_Linear_2_4096_320_out_320 ampere_sgemm_32x128_tn 139.8 1.7E+09 11.0 12.1 78.4 81.8 80.1
unet_block_1 2_Key_Linear_2_4096_320_out_320 ampere_sgemm_32x128_tn 141.6 1.7E+09 11.0 11.9 78.0 82.0 80.2
unet_block_1 3_Value_Linear_2_4096_320_out_320 ampere_sgemm_32x128_tn 140.0 1.7E+09 11.0 12.1 78.6 82.0 80.2
unet_block_1 5-1_MatMul_Query_Key_Then_Scale_Q16_4096_40_KT16_40_4096 ampere_sgemm_128x128_tn 1801.4 2.2E+10 1074.8 12.1 596.6 81.4 48.5
unet_block_1 5-2_Softmax_score_16_4096_4096 cunn_SoftMaxForward 1761.8 8.9E+09 2130.9 5.0 1209.5 60.5 32.3
unet_block_1 5-3_MatMul_Softmax_Value_SM16_4096_4096_V16_4096_40 ampere_sgemm_128x128_nn 4940.0 6.9E+10 1107.2 13.9 224.1 93.1 33.8
unet_block_1 7_Out_Linear_Projection_input2_4096_320 ampere_sgemm_32x128_tn 171.2 1.7E+09 10.9 9.8 63.7 70.7 66.4
unet_block_1 1_Query_Linear_2_4096_320_out_320 ampere_sgemm_32x128_tn 141.6 1.7E+09 11.1 11.9 78.3 82.2 80.4
unet_block_1 2_Key_Linear_2_77_768_out_320 ampere_sgemm_32x32_sliced1x4_tn 1

In [10]:
# Execution_Time_t=0
# Executed_Inst_t=0
# Total_Traffic_t=0

# for kernel in kernels:
#     # range_name = nvtx_info.push_pop_ranges()[6] # 6 Resnet/Transformer Block Level
#     nvtx_domain_ncu_prof = kernel.nvtx_state()[1]
#     nvtx_domain_nsys_prof = kernel.nvtx_state()[2]
#     ncu_range_name = nvtx_domain_ncu_prof.start_end_ranges()[1]
#     nsys_range_name = nvtx_domain_nsys_prof.push_pop_ranges()[-1]
#     if ncu_range_name == "unet_block_1":
#         if nsys_range_name.startswith(yes_list) :
#             if not kernel.name().startswith(no_list) :
#                 # print(range_name)
#                 Work_Achieved = (kernel["smsp__sass_thread_inst_executed_op_fadd_pred_on.sum.per_cycle_elapsed"].value()\
#                                 + kernel["smsp__sass_thread_inst_executed_op_fmul_pred_on.sum.per_cycle_elapsed"].value()\
#                                 + kernel["derived__smsp__sass_thread_inst_executed_op_ffma_pred_on_x2"].value()) \
#                                 * kernel["smsp__cycles_elapsed.avg.per_second"].value()
#                 Traffic_Achieved = kernel["dram__bytes.sum.per_second"].value()                
#                 inst = kernel["thread_inst_executed"].value()
#                 inst2 = kernel["inst_executed"].value()
#                 Execution_Time = kernel["gpu__time_duration.sum"].value()
#                 Executed_Inst = Work_Achieved * Execution_Time * 0.000000001
#                 Total_Traffic = Traffic_Achieved * Execution_Time * 0.000000001
#                 Execution_Time_t += Execution_Time
#                 Executed_Inst_t += Executed_Inst
#                 Total_Traffic_t += Total_Traffic
                
#                 Work_Achieved = Work_Achieved/(1000000000000)
#                 Traffic_Achieved = Traffic_Achieved/(1000000000)
                
#                 SM_BUSY = kernel["sm__throughput.avg.pct_of_peak_sustained_elapsed"].value()
#                 MEM_BUSY = kernel["gpu__compute_memory_access_throughput.avg.pct_of_peak_sustained_elapsed"].value()
                
#                 # print(f"{nsys_range_name} {kernel.name()} {Executed_Inst:.1f} {Total_Traffic/(1000*1000):.2f} , {Execution_Time/1000000}")
#                 print(f"{kernel.name()} {Execution_Time/1000:.1f} {Executed_Inst:.1E} {Total_Traffic/(1000*1000):.1f} {Work_Achieved:.1f} {Traffic_Achieved:.1f} {SM_BUSY:.1f} {MEM_BUSY:.1f}")
#                 # print(f"{Execution_Time_t:.2E}")
#                 # print(f"{Executed_Inst_t:.2E}")
#                 # print(f"{Total_Traffic_t:.2E}")